# Testing Notebook 02:

In [1]:
!pip install ..

from typing import List
import pandas as pd 
import numpy as np

from statdepth.depth import banddepth
from statdepth.depth.depth import _univariate_band_depth, _handle_depth_errors
from statdepth.depth._containment import _select_containment



Processing /Users/julian/Documents/Projects/statdepth
  Created wheel for statdepth: filename=statdepth-0.1.0-py2.py3-none-any.whl size=10741 sha256=30750b0de0ac37d968486abe3a97e0894508a41b5f263f5df091c4549f3ec20a
  Stored in directory: /private/var/folders/pd/jsjcl0fn7w57s5mfr34b20pm0000gn/T/pip-ephem-wheel-cache-wb_kt2z2/wheels/94/78/65/e7cb177a6619a78b187853c88271a8abc224aa673b05e4ed56
Successfully built statdepth
  Attempting uninstall: statdepth
    Found existing installation: statdepth 0.1.0
    Uninstalling statdepth-0.1.0:
      Successfully uninstalled statdepth-0.1.0


In [2]:
!where python

/Users/julian/opt/miniconda3/bin/python
/usr/bin/python


Generate a toy DataFrame for testing

In [3]:
t = [1,2,3,4,4,4,4.5,2,3,2,3]*2
df = pd.DataFrame()
for col in range(15):
    df[col] = t
    
for col in df:
    df[col] = df[col] * np.random.rand() * 2

df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.450518,0.373897,1.889772,1.051792,1.974182,1.856701,0.884417,0.862213,0.622587,1.902597,1.710571,1.589879,1.980816,1.307209,1.987378
1,2.901035,0.747794,3.779544,2.103584,3.948364,3.713401,1.768835,1.724425,1.245174,3.805193,3.421142,3.179758,3.961633,2.614419,3.974755
2,4.351553,1.121692,5.669317,3.155376,5.922547,5.570102,2.653252,2.586638,1.867761,5.707790,5.131712,4.769637,5.942449,3.921628,5.962133
3,5.802070,1.495589,7.559089,4.207168,7.896729,7.426803,3.537669,3.448850,2.490348,7.610387,6.842283,6.359517,7.923265,5.228838,7.949511
4,5.802070,1.495589,7.559089,4.207168,7.896729,7.426803,3.537669,3.448850,2.490348,7.610387,6.842283,6.359517,7.923265,5.228838,7.949511
5,5.802070,1.495589,7.559089,4.207168,7.896729,7.426803,3.537669,3.448850,2.490348,7.610387,6.842283,6.359517,7.923265,5.228838,7.949511
6,6.527329,1.682538,8.503975,4.733064,8.883820,8.355153,3.979878,3.879956,2.801641,8.561685,7.697569,7.154456,8.913674,5.882442,8.943200
7,2.901035,0.747794,3.779544,2.103584,3.948364,3.713401,1.768835,1.724425,1.245174,3.805193,3.421142,3.179758,3.961633,2.614419,3.974755
8,4.351553,1.121692,5.669317,3.155376,5.922547,5.570102,2.653252,2.586638,1.867761,5.707790,5.131712,4.769637,5.942449,3.921628,5.962133
9,2.901035,0.747794,3.779544,2.103584,3.948364,3.713401,1.768835,1.724425,1.245174,3.805193,3.421142,3.179758,3.961633,2.614419,3.974755


In [37]:
def samplebanddepth(data: List[pd.DataFrame], K: int, J=2, containment='r2', relax=False, deep_check=False):
    samples = []

    # Handle common errros
    _handle_depth_errors(data=data, J=J, containment=containment, relax=relax, deep_check=deep_check)

    cdef = _select_containment(containment=containment)
    
    # Univariate case
    if len(data) == 1:
        df = data[0]
        orig = df.copy()
        ss = df.shape[1] // K
        
        for i in range(K):
            depths = []
            t = df.sample(n=ss, axis=1)
            df = df.drop(t.columns, axis=1)

            for col in orig.columns:
                depths.append(_univariate_band_depth(data=t, curve=orig[col], relax=relax, containment=cdef, J=J))

            samples.append(depths)
        print('GOT HERE')
        print(samples)
        samples = pd.Series(index=df.columns, data=[np.mean(i) for i in samples])

    else:
        # Multivariate case: partition list of DataFrames randomly, compute band depth w.r.t those
        shuffled = data.copy()
        random.shuffle(shuffled)
        ss = len(data) // K 

        for _ in range(K):
            pass
        
    return samples

In [16]:
df = pd.DataFrame([
    [1, 2, 3, 6, 9, 8],
    [2, 4, 4, 7, 9, 8],
    [3, 5, 4, 6.5, 12, 10],
    [2, 6, 2, 6, 11, 10],
    [1, 2, 1, 7, 11, 9],
], columns=['f_0', 'f_1', 'f_2', 'f_3', 'f_4', 'f_5'], index=['x_0', 'x_1', 'x_2', 'x_3', 'x_4'])


In [39]:
np.array(samplebanddepth([df], K=2))

GOT HERE
[[0.6666666666666666, 1.0, 0.3333333333333333, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666], [0.0, 0.0, 0.6666666666666666, 1.0, 0.0, 0.6666666666666666]]


ValueError: Length of passed values is 2, index implies 0.

In [20]:
import statdepth

s=statdepth.depth.samplebanddepth([df], K=1)
pd.Series(index=df.columns, data=[np.mean(i) for i in s])

f_0    0.333333
f_1    0.533333
f_2    0.533333
f_3    0.733333
f_4    0.333333
f_5    0.600000
dtype: float64

In [40]:
statdepth.depth.banddepth([df]).sort_values()

f_0    0.333333
f_4    0.333333
f_1    0.533333
f_2    0.533333
f_5    0.600000
f_3    0.733333
dtype: float64